# Analisa Statistik - uji stasioneritas

In [2]:
# pustaka untuk manipulasi data
import pandas as pd
from pandas import concat
from pandas import DataFrame
from pandas import read_csv
from pandas import read_excel
import numpy as np
from numpy import concatenate

# pustaka untuk waktu komputasi
import time
from datetime import datetime

# Pustaka untuk visualisasi data
import seaborn as sns
from matplotlib import pyplot
from matplotlib import pyplot as plt

# Pustaka untuk visualisasi acf dan pacf
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf

# pustaka normalisasi data / membuat data latih dan data uji.
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from mlxtend.preprocessing import minmax_scaling

# pustaka uji stasioneritas statsmodel
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# pustaka uji stasioneritas arch
from arch.unitroot import *
from arch.unitroot import ADF
from arch.unitroot import PhillipsPerron
from arch.unitroot import KPSS

ModuleNotFoundError: No module named 'arch'

### 1. Akuisisi Data

In [ ]:
# Set waktu komputasi
start = time.time()

In [ ]:
# fix random seed for reproducibility
np.random.seed(1234)

In [ ]:
dataset = pd.read_csv("dataset/dataset_boptn.csv", parse_dates=["acq_date"])

In [ ]:
# # set index tanggal
# dataset = dataset.set_index("tanggal")

In [ ]:
dataset.info()

In [ ]:
print(dataset)

- Studi Kasus Sumatera Selatan

In [ ]:
# memilih area studi
df_sumsel = dataset[["hotspot_sumsel"]].copy()
df_sumsel.info()

In [ ]:
print(df_sumsel)

In [ ]:
# code autocorrelation
# membuat frame
fig, ax= plt.subplots(nrows=1, ncols=2, facecolor="#F0F0F0", figsize = (15,4))
plot_acf(df_sumsel, lags=36, ax=ax[0])
plot_pacf(df_sumsel,lags=36, ax=ax[1])
ax[0].grid(True)
ax[1].grid(True)
plt.show()

### 2. Normalisasi Data (Max-Min)

In [ ]:
# memanggil fungsi max min scaler
scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))

In [ ]:
# proses max min scaler
df_norm = scaler.fit_transform(df_sumsel)

In [ ]:
np.round(df_norm[:5],6)

In [ ]:
# generate urutan data sesuai panjang datanya
x = pd.date_range(start="2001-01-01", periods=len(df_sumsel), freq='MS')

# membuat frame
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = (20,5))

# Plot dataset non normalize min-max
# -------------------------------------------------------------------------------
# membuat time series plot
ax1.plot(x, df_sumsel, color="tab:blue", label="data orginal", linewidth=2.5)

# membuat label-label
ax1.set_title("Hotspot Sumsel Sensor MODIS 2001-2020", fontsize=14)
ax1.set_xlabel("Tanggal", fontsize=12)
ax1.set_ylabel("Jumlah Hostpot", fontsize=12)
ax1.legend(loc='upper left')
ax1.grid(True)


# Plot dataset with normalize min-max
# -------------------------------------------------------------------------------
# membuat time series plot
ax2.plot(x, df_norm, color="tab:blue", label="dataset with Normalize Min-Max", linewidth=2.5)

# membuat label-label
ax2.set_title("Hotspot Sumsel Sensor MODIS 2001-2020", fontsize=14)
ax2.set_xlabel("Tanggal", fontsize=12)
ax2.set_ylabel("Jumlah Hostpot", fontsize=12)
ax2.legend(loc='upper left')
ax2.grid(True)

# menampilkan plot
plt.show()

### Uji-Stasioneritas (Statsmodels)

**ADF Test with statsmodel**
- https://machinelearningmastery.com/time-series-data-stationary-python/
- https://docs.w3cub.com/statsmodels/generated/statsmodels.tsa.stattools.adfuller
- https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html

In [ ]:
# Formula ADF Test
# result = adfuller(df_sumsel, maxlag=24, autolag='AIC')
result = adfuller(df_sumsel, autolag=None)

# print result ADF Tes 
print("----------------------------------")
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print(f'n-lags: {result[2]}')
print(f'n-obs: {result[3]}')

print("----------------------------------")
print('Critial Values:')
for key, value in result[4].items():
    print(f'   {key}, {value}')

In [ ]:
if result[1] <= 0.05:
    print('Strong evidence against the null hypothesis (Ho), Reject the null hypothesis, Data has no unit root and is stationary')
else:
    print('Weak evidence against the null hypothesis (Ho), time series has a unit root, indicating it is non stationary.')

### Uji-Stasioneritas (ARCH toolbox)

**ADF Test with ARCH toolbox**
- https://arch.readthedocs.io/en/latest/unitroot/generated/arch.unitroot.ADF.html

Hipotesis 0 (H0): data terdapat akar unit. Artinya data bersifat non-stasioner\
Hipotesis 1 (H1): data terdapat tidak akar unit. Artinya data bersifat stasioner

Jika p-value > 0.05 => Terima H0\
Jika p-value <= 0.05 => Terima H1 

In [ ]:
adf = ADF(df_sumsel, lags=15)
print(adf.summary().as_text())

In [ ]:
# check linier regression with OLS
reg_res = adf.regression
print(reg_res.summary().as_text())

**PP Test with ARCH toolbook**
- https://arch.readthedocs.io/en/latest/unitroot/generated/arch.unitroot.PhillipsPerron.html

Hipotesis 0 (H0): data terdapat akar unit. Artinya data bersifat non-stasioner\
Hipotesis 1 (H1): data terdapat tidak akar unit. Artinya data bersifat stasioner

Jika p-value > 0.05 => Terima H0\
Jika p-value <= 0.05 => Terima H1 

In [ ]:
pp = PhillipsPerron(df_sumsel, lags=15)
print(pp.summary().as_text())

In [ ]:
# # check linier regression with OLS
# reg_res = pp.regression
# print(reg_res.summary().as_text())

**KPSS Test with ARCH toolbook**
- https://arch.readthedocs.io/en/latest/unitroot/generated/arch.unitroot.KPSS.html

Hipotesis 0 (H0): data bersifat stasioner\
Hipotesis 1 (H1): data bersifat non-stasioner

In [ ]:
kpss = KPSS(df_sumsel, lags=15)
print(kpss.summary().as_text())

### Uji-Stasioneritas (Barlett dan Levene) pada nilai ragam

- https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bartlett.html
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.levene.html

In [ ]:
from scipy.stats import bartlett
from scipy.stats import levene
from statistics import mean

In [ ]:
x = dataset["hotspot_sumsel"].values

In [ ]:
def normalize(x):
    return ((x - mean(x)) / (max(x) - min(x)))

In [ ]:
def normalize_1(x, a, b):
    return a + (((x - min(x)) * (b - a)) / (max(x) - min(x)))

In [ ]:
x1 = normalize_1(x, -1, 1)

In [ ]:
# membuat frame
fig, ax1 = plt.subplots(figsize = (20,5))

# Plot dataset non normalize min-max
# -------------------------------------------------------------------------------
# membuat time series plot
ax1.plot(x1, color="tab:blue", label="data orginal", linewidth=2.5)

# membuat label-label
ax1.set_title("Hotspot Sumsel Sensor MODIS 2001-2020", fontsize=14)
ax1.set_xlabel("Tanggal", fontsize=12)
ax1.set_ylabel("Jumlah Hostpot", fontsize=12)
ax1.legend(loc='upper left')
ax1.grid(True)

# menampilkan plot
plt.show()